# Script para gerar a dimensão CALENDARIO
*autor: André Costa (2019-03-09)* [https://www.linkedin.com/in/a-l-costa]


- Data de início: 2000
    - https://www.pcworld.com/article/2000276/a-brief-history-of-gps.html : segundo este artigo, o primeiro dispositivo móvel com serviço de GPS commercial foi o aparelho Benefon Esc!, lançado em 1999 na Europa. Porém, apenas em 2000 o departamento de defesa (dos EUA) removeu uma restrição de degradação do sinal de GPS, que fora implementada por interesse militar, possibilitando o desenvolvimento da tecnologia. Portanto, em uma estimativa conservadora, considerando também o cenário de desenvolvimento da tecnologia na localidade escopo da aplicação, acredita-se que a chance de haver um evento de fluxo com data anterior ao ano 2000 é ínfima.

- Data de expiração: 2030
    - Embora a data de expiração do calendário seja consideravelmente curta, com aproximadamente 10 anos de vida útil, esta dimensão pode ser facilmente atualizada em qualquer momento, com custo irrelevante. Recomendamos disparar um evento, com 30 dias de antecedência, pelos processos internos da empresa, para notificar sobre a necessidade de executar esta tarefa, caso o sistema esteja ainda em operação.


## Gênesis

- Esta etapa realiza a geração dos dados que irão popular a **dimensão calendário**, utilizando uma função personalizada e uma serie de execuções dentro do contexto SPARK
    1. Uma sequencia de datas é gerada utilizando a biblioteca Pandas
    2. A sequência de datas é manipulada para gerar os registros, por meio da função *gen_calendar_records*, onde para cada dia na sequência de dados, três registros da **dimensão calendário** são gerados, uma para cada período do dia [manhã, tarde e noite]
    

In [1]:
import pyspark.sql as pysql
import pandas as pd

translation = {'Sunday': 0, 
               'Monday': 1,
               'Tuesday': 2,
               'Wednesday': 3,
               'Thursday': 4,
               'Friday': 5,
               'Saturday': 6,
              }

start_date = '2000-01-01'
BIG_BANG = pd.Timestamp(start_date)
def gen_calendar_records(date):
    dt = pd.Timestamp(date)
    days_offset = (dt - BIG_BANG).days * 3
    rows = []
    for i in range(3):
        rows.append(pysql.Row(id=days_offset+i, ano=dt.year, mes=dt.month,
                              dia=dt.day, periodo=i, dia_semana=translation[dt.day_name()]))
    return rows

end_date = '2029-12-31'

# Gera sequência de datas
dates = pd.date_range(start_date, end_date)

seed_rdd = sc.parallelize(dates)

# Aplica a transformação que cria os registros do calendário
gen_rdd = seed_rdd.flatMap(gen_calendar_records)
print(gen_rdd.take(5))

[Row(ano=2000, dia=1, dia_semana=6, id=0, mes=1, periodo=0), Row(ano=2000, dia=1, dia_semana=6, id=1, mes=1, periodo=1), Row(ano=2000, dia=1, dia_semana=6, id=2, mes=1, periodo=2), Row(ano=2000, dia=2, dia_semana=0, id=3, mes=1, periodo=0), Row(ano=2000, dia=2, dia_semana=0, id=4, mes=1, periodo=1)]


## Materialização

- Nesta etapa, o RDD é convertido em uma outra estrutura, o *Data-Frama*, conectando a tabela de dados ao esquema de dados, no processo


In [2]:
from pyspark.sql.types import *
calendarSchema = StructType([
             StructField('id', IntegerType()),
             StructField('ano', IntegerType()),
             StructField('mes', ShortType()),
             StructField('dia', ShortType()),
             StructField('periodo', ShortType()),
             StructField('dia_semana', ShortType())
            ])

calendar = sqlContext.createDataFrame(gen_rdd, calendarSchema)
calendar.printSchema()

root
 |-- id: integer (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: short (nullable = true)
 |-- dia: short (nullable = true)
 |-- periodo: short (nullable = true)
 |-- dia_semana: short (nullable = true)



## Persistência

- Nesta etapa, os dados estruturados são armazenados dentro do sistema de arquivos do hadoop, e gerenciados pelo serviço **HIVE**, em formato de arquivo ORC

    1. Este script assume que os dados serão sobrescritos, e não incrementados. Assim, a primeira etapa é limpar todos os dados da tabela usando a função *truncate*
    2. Então, uma tabela temporária do HIVE é criada com os dados estruturados
    3. Finalmente, todos os registros da tabela temporária são copiados para a tabela definitiva


In [3]:
spark.sql('use geodata')
spark.sql('SET hive.exec.dynamic.partition=true')
spark.sql('SET hive.exec.dynamic.partition.mode=nonstrict')
spark.sql('create table if not exists calendario_buff like calendario')
calendar.createOrReplaceTempView('temp_calendar')
spark.sql('insert into table calendario_buff PARTITION (dia_semana) (select * from temp_calendar)')
spark.sql('drop table calendario')
spark.sql('alter table calendario_buff RENAME TO calendario')
spark.sql('select * from calendario limit 10').show()

+-----+----+---+---+-------+----------+
|   id| ano|mes|dia|periodo|dia_semana|
+-----+----+---+---+-------+----------+
|15372|2014|  1| 11|      0|         6|
|15373|2014|  1| 11|      1|         6|
|15374|2014|  1| 11|      2|         6|
|15393|2014|  1| 18|      0|         6|
|15394|2014|  1| 18|      1|         6|
|15395|2014|  1| 18|      2|         6|
|15414|2014|  1| 25|      0|         6|
|15415|2014|  1| 25|      1|         6|
|15416|2014|  1| 25|      2|         6|
|15435|2014|  2|  1|      0|         6|
+-----+----+---+---+-------+----------+

